<a href="https://colab.research.google.com/github/benjaminbrown038/ECSE-6410-Pattern-Recognition-HW1/blob/main/ECSE_6410_Pattern_Recognition_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 - Classify Cats and Dogs


# Imports

In [10]:

import tensorflow as tf
import matplotlib.pyplot as plt
import os
import zipfile

# Data




In [3]:
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path = tf.keras.utils.get_file("cats_and_dogs.zip", origin=url, extract=True)


In [4]:
base_dir = os.path.join(os.path.dirname(path), 'cats_and_dogs_filtered')


In [5]:
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'validation')


In [8]:
train_dir

'/root/.keras/datasets/cats_and_dogs_filtered/train'

In [13]:


url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=url, extract=True)
train_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered', 'train')

print(f"Train directory: {path_to_data}")



Train directory: /root/.keras/datasets/cats_and_dogs_filtered/train


In [12]:
path_to_data

'/root/.keras/datasets/cats_and_dogs_filtered/train'

In [6]:
img_size = (150, 150)
batch_size = 32

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True)

NotFoundError: Could not find directory /root/.keras/datasets/cats_and_dogs_filtered/train

In [2]:
# -----------------------------
# 1. Download and Prepare Dataset
# -----------------------------
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path = tf.keras.utils.get_file("cats_and_dogs.zip", origin=url, extract=True)

base_dir = os.path.join(os.path.dirname(path), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'validation')


img_size = (150, 150)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


NotFoundError: Could not find directory /root/.keras/datasets/cats_and_dogs_filtered/train

In [ ]:
print("Train Dir Exists:", os.path.exists(train_dir))
print("Train Dir Content:", os.listdir(train_dir))

Train Dir Exists: False


FileNotFoundError: [Errno 2] No such file or directory: '/root/.keras/datasets/cats_and_dogs_filtered/train'

In [ ]:
# -----------------------------
# 2. Create Training and Validation Datasets
# -----------------------------
img_size = (150, 150)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)





NotFoundError: Could not find directory /root/.keras/datasets/cats_and_dogs_filtered/train

In [ ]:
# Improve performance with prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# -----------------------------
# 3. Data Augmentation Layer
# -----------------------------
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),])


# Model



In [ ]:
# -----------------------------
# 4. Define CNN Model
# -----------------------------
model = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255, input_shape=(150, 150, 3)),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  ])

In [ ]:
# -----------------------------
# 5. Compile Model
# -----------------------------
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# -----------------------------
# 6. Train Model
# -----------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10)

In [ ]:
# -----------------------------
# 7. Plot Training Results
# -----------------------------
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend()
plt.title("Loss")
plt.show()

### Dataset Reference

1. https://www.kaggle.com/c/dogs-vs-cats/data